In [1]:
import pandas as pd
import sys
import os
import anndata as ad
import pandas as pd
import scanpy as sc
import seaborn as sns; sns.set(color_codes=True)
import numpy as np
# Import Scimap
import scimap as sm
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad


Running SCIMAP  2.2.7


Process files

In [15]:
def gather_txt_files_by_folder(main_folder):
    txt_files = []
    for root, dirs, files in os.walk(main_folder):
        for file in files:
            if file.endswith('.csv'):
                txt_files.append(os.path.join(root, file))
    return txt_files

# Specify the main folder to search in
main_folder_path = "/TMA_reduced_quantified_bcl6_five/"  # Replace with your folder path
txt_file_list = gather_txt_files_by_folder(main_folder_path)

In [16]:
for txt_file in txt_file_list:
    print(txt_file)

# Printing the count of different txt files
print(f"Number of different .csv files: {len(txt_file_list)}")

/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/801_GC1.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/801_GC2.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/803_GC1.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/803_GC2.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/805_GC1.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/807_GC1.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/807_GC2.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/809_GC1.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/809_GC2.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/811_GC1.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/811_GC2.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/813_GC1.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_fiv

In [ ]:
# dividing to four categories did not look too good. Lets divide now into 5 categories. Inspect the results later.
for txt_file in txt_file_list:
    df = pd.read_csv(txt_file, sep=',')
    markers = ['DAP1', 'HEV', 'FOXP3', 'ICOS', 'CD27', 'DAPI2',
       'CD19', 'BCL6', 'DAPI3', 'CD4', 'CXCR5', 'PD1', 'DAPI4', 'KI67', 'CD3',
       'CD8', 'DAPI5', 'AE1_3', 'CD21', 'DAPI6', 'CD20']
    # Create a new DataFrame to store the processed results
    for marker in markers:
        if marker in df.columns:
            df[marker] = np.log1p(df[marker])
    new_full_df_nmm = df
    markers = ['BCL6','CD27']
    # Loop through each marker
    for marker in markers:
        status_col = f'{marker}_status_4'  # Generate a status column name based on the marker. okay so forgot to change this for "five"...
        new_full_df_nmm[status_col] = np.nan  # Initialize the new column
        all_annotated_labels = []  # Create an empty list to hold all the annotated labels
        subset_data = new_full_df_nmm
        data = subset_data[marker].values.reshape(-1, 1)

        # Initialize and fit the GMM with 5 components
        gmm = GaussianMixture(n_components=5, random_state=0)
        gmm.fit(data)
        labels = gmm.predict(data)
        
        label_df = pd.DataFrame({"label": labels, marker: subset_data[marker].values})
        medians = label_df.groupby("label")[marker].median().sort_values().index

        # Map the labels to the descriptive categories
        status_labels = ["bg", "neg", "low", "med", "hi"]
        label_mapping = {medians[i]: status_labels[i] for i in range(len(medians))}
        annotated_labels = [label_mapping[label] for label in labels]
        all_annotated_labels.extend(annotated_labels)

        # Assign the annotated labels back to the corresponding status column in the DataFrame
        new_full_df_nmm[status_col] = all_annotated_labels

    status_mapping = {
        'bg': 0,
        'neg': 1,
        'low': 2,
        'med': 3,
        'hi': 4}
    
    # List of columns ending with '_status'
    status_columns = [col for col in new_full_df_nmm.columns if col.endswith('_status_5')]
    df = new_full_df_nmm
    # Apply the mapping to the relevant columns
    for col in status_columns:
        new_full_df_nmm[col] = new_full_df_nmm[col].replace(status_mapping)
    classification_data = new_full_df_nmm[status_columns]
    modified_file_path = txt_file.replace('.csv', '_bcl6status.csv')  # Change '.txt' to '.csv' if necessary
    print(modified_file_path)
    new_full_df_nmm.to_csv(modified_file_path, sep=',', index=False)


/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/801_GC1_bcl6status.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/801_GC2_bcl6status.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/803_GC1_bcl6status.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/803_GC2_bcl6status.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/805_GC1_bcl6status.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/807_GC1_bcl6status.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/807_GC2_bcl6status.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/809_GC1_bcl6status.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/809_GC2_bcl6status.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/811_GC1_bcl6status.csv
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/811_GC2_bcl6status.csv
/Volumes/Kek_2Tt_7T/T

In [12]:
print(txt_file)

/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified/327_GC1.csv


In [36]:
def gather_txt_files_by_folder(main_folder):
    txt_files = []
    for root, dirs, files in os.walk(main_folder):
        for file in files:
            if file.endswith('.txt'):
                txt_files.append(os.path.join(root, file))
    return txt_files

# Specify the main folder to search in
main_folder_path = "/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/"  # Replace with your folder path
txt_file_list = gather_txt_files_by_folder(main_folder_path)

In [37]:
for txt_file in txt_file_list:
    print(txt_file)

# Printing the count of different txt files
print(f"Number of different .csv files: {len(txt_file_list)}")

/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/115_GC1/quantification/115_GC1.txt
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/105_GC1/quantification/105_GC1.txt
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/105_GC2/quantification/105_GC2.txt
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/107_GC1/quantification/107_GC1.txt
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/107_GC2/quantification/107_GC2.txt
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/109_GC2/quantification/109_GC2.txt
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/111_GC1/quantification/111_GC1.txt
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/125_GC2/quantification/125_GC2.txt
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/117_GC1/quantification/117_GC1.txt
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/117_GC2/quantification/117_GC2.txt
/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/119_GC2/quantification/119_GC2.txt

In [38]:
for txt_file in txt_file_list:
    df = pd.read_csv(txt_file, sep=',')
    markers = ['DAP1', 'HEV', 'FOXP3', 'ICOS', 'CD27', 'DAPI2',
       'CD19', 'BCL6', 'DAPI3', 'CD4', 'CXCR5', 'PD1', 'DAPI4', 'KI67', 'CD3',
       'CD8', 'DAPI5', 'AE1_3', 'CD21', 'DAPI6', 'CD20']
    # Create a new DataFrame to store the processed results
    for marker in markers:
        if marker in df.columns:
            df[marker] = np.log1p(df[marker])
    new_full_df_nmm = df
    markers = ['BCL6','CD27']
    # Loop through each marker
    for marker in markers:
        status_col = f'{marker}_status_4'  # Generate a status column name based on the marker
        new_full_df_nmm[status_col] = np.nan  # Initialize the new column
        all_annotated_labels = []  # Create an empty list to hold all the annotated labels
        subset_data = new_full_df_nmm
        data = subset_data[marker].values.reshape(-1, 1)

        # Initialize and fit the GMM with 4 components
        gmm = GaussianMixture(n_components=4, random_state=0)
        gmm.fit(data)
        labels = gmm.predict(data)
        
        label_df = pd.DataFrame({"label": labels, marker: subset_data[marker].values})
        medians = label_df.groupby("label")[marker].median().sort_values().index

        # Map the labels to the descriptive categories
        status_labels = ["bg", "low", "med", "hi"]
        label_mapping = {medians[i]: status_labels[i] for i in range(len(medians))}
        annotated_labels = [label_mapping[label] for label in labels]
        all_annotated_labels.extend(annotated_labels)

        # Assign the annotated labels back to the corresponding status column in the DataFrame
        new_full_df_nmm[status_col] = all_annotated_labels

    status_mapping = {
        'bg': 0,
        'low': 1,
        'med': 2,
        'hi': 3 }
    
    # List of columns ending with '_status'
    status_columns = [col for col in new_full_df_nmm.columns if col.endswith('_status_4')]
    df = new_full_df_nmm
    # Apply the mapping to the relevant columns
    for col in status_columns:
        new_full_df_nmm[col] = new_full_df_nmm[col].replace(status_mapping)
    classification_data = new_full_df_nmm[status_columns]
    modified_file_path = txt_file.replace('.txt', '_bcl6status.csv')  # Change '.txt' to '.csv' if necessary
    print(modified_file_path)
    new_full_df_nmm.to_csv(modified_file_path, sep=',', index=False)


/var/folders/02/vkyq99757hsfjng_p8wh1wkc0bfbtx/T/ipykernel_96970/1421698353.py:48: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/115_GC1/quantification/115_GC1_bcl6status.csv


/var/folders/02/vkyq99757hsfjng_p8wh1wkc0bfbtx/T/ipykernel_96970/1421698353.py:48: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/105_GC1/quantification/105_GC1_bcl6status.csv


/var/folders/02/vkyq99757hsfjng_p8wh1wkc0bfbtx/T/ipykernel_96970/1421698353.py:48: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



...

/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/306_GC1/quantification/306_GC1_bcl6status.csv


/var/folders/02/vkyq99757hsfjng_p8wh1wkc0bfbtx/T/ipykernel_96970/1421698353.py:48: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/315_GC1/quantification/315_GC1_bcl6status.csv


/var/folders/02/vkyq99757hsfjng_p8wh1wkc0bfbtx/T/ipykernel_96970/1421698353.py:48: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/315_GC2/quantification/315_GC2_bcl6status.csv


In [11]:
new_full_df_nmm

AttributeError: 'DataFrame' object has no attribute 'obs'

In [10]:
###CHECKING on images
marker_list_with_DNA = ['DAP1', 'HEV', 'FOXP3', 'ICOS', 'CD27', 'DAPI2',
       'CD19', 'BCL6', 'DAPI3', 'CD4', 'CXCR5', 'PD1', 'DAPI4', 'KI67', 'CD3',
       'CD8', 'DAPI5', 'AE1_3', 'CD21', 'DAPI6', 'CD20']

data = new_full_df_nmm[marker_list_with_DNA]
meta = new_full_df_nmm.drop(columns=marker_list_with_DNA)
#Remove DNA channels
hoechst_columns = [col for col in data.columns if "DNA" in col]
data.drop(columns=hoechst_columns, inplace=True)
columns = data.columns
data = np.log1p(data[data.columns])
adata = ad.AnnData (data)
adata.obs = meta
directory_img = "/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_thy_stacksandmasks/327_GC1/"
sample = "327_GC1"
file_image_dir = directory_img + sample + ".tif"
sample_data=adata
sm.pl.image_viewer(file_image_dir, sample_data, overlay = 'BCL6_status_4', channel_names = marker_list_with_DNA,x_coordinate="X_centroid",y_coordinate="Y_centroid", point_size=10)


/var/folders/02/vkyq99757hsfjng_p8wh1wkc0bfbtx/T/ipykernel_96970/3764401669.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jssarkki/opt/anaconda3/envs/scimap/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning:

Transforming to str index.



In [16]:
adata.obs

,CellID,Y_centroid,X_centroid,Area,Eccentricity,MajorAxisLength,MinorAxisLength,Solidity,Extent,BCL6_status_4,CD27_status_4
0,1,2308.208333,2325.125000,96.0,0.808494,NaN,NaN,NaN,NaN,1,2
1,2,1438.623762,2317.698020,202.0,0.824560,NaN,NaN,NaN,NaN,2,2
2,3,4776.455446,2374.099010,202.0,0.709135,NaN,NaN,NaN,NaN,0,0
3,4,4212.446009,2758.140845,213.0,0.768157,NaN,NaN,NaN,NaN,0,0
4,5,4501.756000,3732.000000,250.0,0.658651,NaN,NaN,NaN,NaN,2,3
...,...,...,...,...,...,...,...,...,...,...,...
41245,41246,3774.583784,2491.362162,185.0,0.772442,NaN,NaN,NaN,NaN,0,1
41246,41247,3057.119048,2733.126984,126.0,0.683777,NaN,NaN,NaN,NaN,0,1
41247,41248,2883.363636,4438.909091,11.0,0.708444,NaN,NaN,NaN,NaN,1,3
41248,41249,3723.647059,5105.058824,17.0,0.875727,NaN,NaN,NaN,NaN,3,3


Check the outcome

In [2]:
redu5 = pd.read_csv('/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_reduced_quantified_bcl6_five/636_GC1_bcl6status.csv', sep=',')
og = pd.read_csv('/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_ms_stacksandmasks/636_GC1/quantification/636_GC1_bcl6status.csv', sep=',')

In [3]:
print(redu5)
print(og)

       CellID   Y_centroid   X_centroid   Area  Eccentricity      DAP1  \
0           1  2391.313559  4322.135593  354.0      0.817441  8.499679   
1           2  3581.918103  1860.827586  232.0      0.599276  8.986815   
2           3  2034.432432   689.085586  222.0      0.745140  8.810263   
3           4  3184.148148  1008.722222   54.0      0.364380  8.889630   
4           5  1212.988142  3063.715415  253.0      0.506180  8.739803   
...       ...          ...          ...    ...           ...       ...   
23951   23952  4114.500000  1333.500000    8.0      0.939700  9.174169   
23952   23953  2971.810345  2497.482759   58.0      0.812355  8.338800   
23953   23954  3514.000000  1202.200000   15.0      0.817798  9.401490   
23954   23955  2898.800000  2932.000000   25.0      0.887009  7.752541   
23955   23956  2010.540404  4005.813131  198.0      0.889303  9.248633   

            HEV     FOXP3      ICOS      CD27  ...     AE1_3      CD21  \
0      6.302634  5.394706  7.230682  

In [5]:
###CHECKING on images
marker_list_with_DNA = ['DAP1', 'HEV', 'FOXP3', 'ICOS', 'CD27', 'DAPI2',
       'CD19', 'BCL6', 'DAPI3', 'CD4', 'CXCR5', 'PD1', 'DAPI4', 'KI67', 'CD3',
       'CD8', 'DAPI5', 'AE1_3', 'CD21', 'DAPI6', 'CD20']

data = redu5[marker_list_with_DNA]
meta = redu5.drop(columns=marker_list_with_DNA)
#Remove DNA channels
hoechst_columns = [col for col in data.columns if "DNA" in col]
data.drop(columns=hoechst_columns, inplace=True)
columns = data.columns
data = np.log1p(data[data.columns])
adata = ad.AnnData (data)
adata.obs = meta
directory_img = "/636_GC1/"
sample = "636_GC1"
file_image_dir = directory_img + sample + ".tif"
sample_data=adata
sm.pl.image_viewer(file_image_dir, sample_data, overlay = 'BCL6_status_4', channel_names = marker_list_with_DNA,x_coordinate="X_centroid",y_coordinate="Y_centroid", point_size=10)


/var/folders/02/vkyq99757hsfjng_p8wh1wkc0bfbtx/T/ipykernel_50250/3076596209.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jssarkki/opt/anaconda3/envs/scimap/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning:

Transforming to str index.

/Users/jssarkki/opt/anaconda3/envs/scimap/lib/python3.10/site-packages/napari/_qt/widgets/qt_dims.py:309: UserWarning:

Refusing to play a hidden axis



In [6]:
###CHECKING on images
marker_list_with_DNA = ['DAP1', 'HEV', 'FOXP3', 'ICOS', 'CD27', 'DAPI2',
       'CD19', 'BCL6', 'DAPI3', 'CD4', 'CXCR5', 'PD1', 'DAPI4', 'KI67', 'CD3',
       'CD8', 'DAPI5', 'AE1_3', 'CD21', 'DAPI6', 'CD20']

data = og[marker_list_with_DNA]
meta = og.drop(columns=marker_list_with_DNA)
#Remove DNA channels
hoechst_columns = [col for col in data.columns if "DNA" in col]
data.drop(columns=hoechst_columns, inplace=True)
columns = data.columns
data = np.log1p(data[data.columns])
adata = ad.AnnData (data)
adata.obs = meta
directory_img = "/Volumes/Kek_2Tt_7T/TMA_analysis/TMA_ms_stacksandmasks/636_GC1/"
sample = "636_GC1"
file_image_dir = directory_img + sample + ".tif"
sample_data=adata
sm.pl.image_viewer(file_image_dir, sample_data, overlay = 'BCL6_status_4', channel_names = marker_list_with_DNA,x_coordinate="X_position",y_coordinate="Y_position", point_size=10)


/var/folders/02/vkyq99757hsfjng_p8wh1wkc0bfbtx/T/ipykernel_50250/2060526039.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jssarkki/opt/anaconda3/envs/scimap/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning:

Transforming to str index.



In [ ]:
###CHECKING on images
marker_list_with_DNA = ['DAP1', 'HEV', 'FOXP3', 'ICOS', 'CD27', 'DAPI2',
       'CD19', 'BCL6', 'DAPI3', 'CD4', 'CXCR5', 'PD1', 'DAPI4', 'KI67', 'CD3',
       'CD8', 'DAPI5', 'AE1_3', 'CD21', 'DAPI6', 'CD20']

data = new_full_df_nmm[marker_list_with_DNA]
meta = new_full_df_nmm.drop(columns=marker_list_with_DNA)
#Remove DNA channels
hoechst_columns = [col for col in data.columns if "DNA" in col]
data.drop(columns=hoechst_columns, inplace=True)
columns = data.columns
data = np.log1p(data[data.columns])
adata = ad.AnnData (data)
adata.obs = meta
directory_img = "D://joona//853//"
sample = "853_GC1"
file_image_dir = directory_img + sample + ".tif"
sample_data=adata
sm.pl.image_viewer(file_image_dir, sample_data, overlay = 'BCL6_status', channel_names = marker_list_with_DNA,x_coordinate="X_position",y_coordinate="Y_position", point_size=10)


C:\Users\Localadmin_junqmada\AppData\Local\Temp\ipykernel_24500\2385991462.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Localadmin_junqmada\AppData\Roaming\Python\Python310\site-packages\anndata\_core\anndata.py:120: ImplicitModificationWarning:

Transforming to str index.

